In [24]:
import os
infected_path = os.path.join(
    'cell_images', 'Parasitized')
uninfected_path = os.path.join(
    'cell_images', 'Uninfected')

infected = os.listdir(infected_path)
uninfected = os.listdir(uninfected_path)

In [25]:
infected_path, uninfected_path

('cell_images\\Parasitized', 'cell_images\\Uninfected')

In [26]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import os
import cv2
import seaborn as sns

In [27]:
import matplotlib
matplotlib.use("TkAgg")

In [28]:
data = []
labels = []


for i in infected:
    try:
        image = cv2.imread(
            'cell_images\\Parasitized\\'+i)
        image_array = Image.fromarray(image, 'RGB')
        resize_img = image_array.resize((50, 50))
        rotated45 = resize_img.rotate(45)
        rotated75 = resize_img.rotate(75)
        blur = cv2.blur(np.array(resize_img), (10, 10))
        data.append(np.array(resize_img))
        data.append(np.array(rotated45))
        data.append(np.array(rotated75))
        data.append(np.array(blur))
        labels.append(1)
        labels.append(1)
        labels.append(1)
        labels.append(1)
    except AttributeError:
        print('error')

print("infected done")

for u in uninfected:
    try:
        image = cv2.imread(
            'cell_images\\Uninfected\\'+u)
        image_array = Image.fromarray(image, 'RGB')
        resize_img = image_array.resize((50, 50))
        rotated45 = resize_img.rotate(45)
        rotated75 = resize_img.rotate(75)
        data.append(np.array(resize_img))
        data.append(np.array(rotated45))
        data.append(np.array(rotated75))
        labels.append(0)
        labels.append(0)
        labels.append(0)
    except AttributeError:
        print('error')

error
infected done
error


In [29]:
cells = np.array(data)
labels = np.array(labels)

In [30]:
plt.figure(1, figsize=(15, 9))
n = 0
for i in range(14):
    n += 1
    r = np.random.randint(0, cells.shape[0], 1)
    plt.subplot(7, 7, n)
    plt.subplots_adjust(hspace=0.5, wspace=0.5)
    plt.imshow(cells[r[0]])
    plt.title('{} : {}'.format('Infected' if labels[r[0]] == 1 else 'Unifected',
                               labels[r[0]]))
    plt.xticks([]), plt.yticks([])

plt.show()

In [31]:
# Normalizing and Randomize order of data

n = np.arange(cells.shape[0])
np.random.shuffle(n)
cells = cells[n]
labels = labels[n]
cells = cells.astype(np.float32)
labels = labels.astype(np.int32)
cells = cells/255

In [33]:
from sklearn.model_selection import train_test_split

train_x, x, train_y, y = train_test_split(cells, labels,
                                          test_size=0.2,
                                          random_state=111)

eval_x, test_x, eval_y, test_y = train_test_split(x, y,
                                                  test_size=0.5,
                                                  random_state=111)

In [34]:
print('train data shape {} ,eval data shape {} , test data shape {}'.format(train_x.shape,
                                                                            eval_x.shape,
                                                                            test_x.shape))

train data shape (77162, 50, 50, 3) ,eval data shape (9645, 50, 50, 3) , test data shape (9646, 50, 50, 3)


In [35]:
tf.keras.backend.clear_session()

In [36]:
import keras
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(50, 50, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.BatchNormalization(axis = -1),
    keras.layers.Dropout(0.2),
    keras.layers.Conv2D(32, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.BatchNormalization(axis = -1),
    keras.layers.Dropout(0.2),
    keras.layers.Flatten(),
    keras.layers.Dense(activation='relu', units = 512),
    keras.layers.BatchNormalization(axis = -1),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(activation='relu', units = 256),
    keras.layers.BatchNormalization(axis = -1),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(activation='sigmoid', units = 1)
])

model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['accuracy'])
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 24, 32)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 24, 24, 32)       128       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 24, 24, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 11, 11, 32)       0

In [37]:
history = model.fit(train_x, train_y, epochs=3, validation_data=(test_x, test_y))

Epoch 1/3
2412/2412 [==============================] - 224s 92ms/step - loss: 0.2208 - accuracy: 0.9135 - val_loss: 0.1524 - val_accuracy: 0.9485
Epoch 2/3
2412/2412 [==============================] - 247s 102ms/step - loss: 0.1316 - accuracy: 0.9553 - val_loss: 0.2101 - val_accuracy: 0.9233
Epoch 3/3
2412/2412 [==============================] - 191s 79ms/step - loss: 0.1216 - accuracy: 0.9587 - val_loss: 0.1409 - val_accuracy: 0.9520


In [38]:
test_loss, test_acc = model.evaluate(test_x, test_y, verbose=2)
predictions = model.predict(test_x)

302/302 - 3s - loss: 0.1409 - accuracy: 0.9520 - 3s/epoch - 11ms/step
302/302 [==============================] - 4s 12ms/step


In [39]:
predictions = (predictions >= 0.5).astype(int)
predictions

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [1]])

In [40]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_y, predictions)
cm

array([[3963,  127],
       [ 336, 5220]], dtype=int64)

In [41]:
from sklearn.metrics import accuracy_score,precision_score
accuracy_score(test_y,predictions), precision_score(test_y,predictions)

(0.95200082935932, 0.9762483635683561)

In [ ]:
plt.figure(1 , figsize = (15 , 9))
n = 0 
for i in range(28):
    n += 1 
    r = np.random.randint( 0  , test_x.shape[0] , 1)
    plt.subplot(7 , 7 , n)
    plt.subplots_adjust(hspace = 0.5 , wspace = 0.5)
    plt.imshow(test_x[r[0]])
    plt.title('true {} : pred {}'.format(test_y[r[0]] , predictions[r[0]]) )
    plt.xticks([]) , plt.yticks([])

plt.show()